# Step 0 - Load dependencies

In [12]:
import os
import sentencepiece as sp
from datasets import list_datasets
from datasets import load_dataset
import pandas as pd

/root/miniconda3/envs/learntosearch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step 1 - Load the datasets

In [11]:
datasets_list = list_datasets()
ms_df_dict = load_dataset("ms_marco", "v1.1")
ms_train_df = ms_df_dict['train']
ms_validation_df = ms_df_dict['validation']
ms_test_df = ms_df_dict['test']

# Convert to Pandas
ms_train_df = pd.DataFrame(ms_train_df)
ms_validation_df = pd.DataFrame(ms_validation_df)
ms_test_df = pd.DataFrame(ms_test_df)

print(ms_train_df.head())

                                             answers  ... wellFormedAnswers
0  [Results-Based Accountability is a disciplined...  ...                []
1                                              [Yes]  ...                []
2                                    [20-25 minutes]  ...                []
3                       [$11 to $22 per square foot]  ...                []
4                      [Due to symptoms in the body]  ...                []

[5 rows x 6 columns]


Setup a smaller dataset for debugging purposes

In [13]:
ms_train_df_short = ms_train_df.iloc[0:1000]
ms_validation_df_short = ms_validation_df.iloc[0:600]
ms_test_df_short = ms_test_df[0:200]

# Step X - Setup sentence piece

In [13]:
corpus = "Testing"

corpus_filename = "./datasets/ms_marco_corpus.txt"
with open(corpus_filename, "w") as corpus_file:
    for sentence in corpus:
        corpus_file.write(sentence + os.linesep)